# MapReduce

## ... for pedestrians

In [ ]:
def square(x):
    return x**2

In [ ]:
map(square, range(10))

In [ ]:
[x**2 for x in range(10)]

In [ ]:
map(lambda x: x**2, range(10))

In [ ]:
def my_add(x, y):
    return x + y

In [ ]:
reduce(my_add, range(10))

## My first MapReduce

In [ ]:
reduce(my_add, map(square, range(10)))

### but what about the keys?

In [ ]:
pairs = map(lambda x: (x % 2 == 0, x**2 + x), range(10))
pairs

In [ ]:
from collections import defaultdict
def reduceByKey(reduce_fn, iterable):
    chunks = defaultdict(list)
    for k, v in iterable:
        chunks[k].append(v)
    for k in chunks:
        chunks[k] = reduce(reduce_fn, chunks[k])
    return chunks

In [ ]:
reduceByKey(my_add, pairs)

## Staring Spark Locally

    PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ~/src/spark-2.1.0-bin-hadoop2.7/bin/pyspark 

## The spark context

In [ ]:
sc

## Baby steps

In [ ]:
RDD = sc.parallelize([1,2,3,4,5])

In [ ]:
RDD

In [ ]:
RDD.collect()

In [ ]:
RDD.count()

In [ ]:
RDD.map(lambda x: x**2).collect()

## Loading data

In [ ]:
wine = sc.textFile('data/winequality-white.csv')

In [ ]:
wine.take(5)

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
obama_url = 'https://en.wikipedia.org/wiki/Barack_Obama'
obama_soup = BeautifulSoup(requests.get(obama_url).text, 'lxml')
for unwanted in obama_soup(['script', 'style']):
    unwanted.decompose()
obama_text = obama_soup.get_text()

In [ ]:
obamaRDD = sc.parallelize(obama_text.split())

In [ ]:
obamaRDD.take(5)

## Basic RDD operations

In [ ]:
word_lengths = obamaRDD.map(lambda x: len(x))

In [ ]:
word_lengths.take(5)

In [ ]:
import re

In [ ]:
clean_RDD = obamaRDD.filter(lambda x: re.match('\w+', x))

In [ ]:
clean_RDD.take(5)

In [ ]:
word_lengths = clean_RDD.map(lambda x: len(x))

In [ ]:
word_count = word_lengths.count()
word_count

In [ ]:
text_length = word_lengths.reduce(lambda x, y: x + y)

In [ ]:
text_length

In [ ]:
# mean word lenght
text_length / float(word_count)

In [ ]:
from operator import add

In [ ]:
add(1,2)

In [ ]:
word_lengths.reduce(add)

In [ ]:
word_counts = (
    obamaRDD
      .map(lambda x: (x, 1))
      .reduceByKey(add))

In [ ]:
word_counts.take(5)

In [ ]:
word_counts.takeOrdered(5, key=lambda (_, v): -v)

In [ ]:
(word_counts
 .map(lambda (k, v): (k.lower(), v))
 .filter(lambda (k, _): re.match('\w+', k))
 .reduceByKey(add)
 .takeOrdered(5, key=lambda (_, v): -v))

In [ ]:
class DontTryThisAtHome(object):
    def my_add(self, a, b):
        return a + b + 1
    def sketchy_top_words(self, rdd, n):
        return (rdd
                .map(lambda x: (x, 1))
                .reduceByKey(self.my_add)
                .takeOrdered(n, key=lambda (_, v): -v))

In [ ]:
DontTryThisAtHome().sketchy_top_words(obamaRDD, 10)

In [ ]:
counter = 0
def my_count(_):
    global counter
    counter += 1

In [ ]:
obamaRDD.foreach(my_count)

In [ ]:
counter

In [ ]:
acc = sc.accumulator(0)

In [ ]:
obamaRDD.foreach(lambda _: acc.add(1))

In [ ]:
acc.value

## Monitor Performance

http://localhost:4040/jobs/

## Set-like Operations

In [ ]:
a = sc.parallelize([1,2,3,4,4])
b = sc.parallelize([3,4,5])

In [ ]:
a.distinct().collect()

In [ ]:
a.union(b).collect()

In [ ]:
a.intersection(b).collect()

In [ ]:
a.cartesian(b).collect()

## Other Operations of note

In [ ]:
sc.parallelize(range(100)).sample(True, 0.1).collect()

In [ ]:
a = sc.parallelize([('foo', 1), ('bar', 1)])
b = sc.parallelize([('bar', 2), ('baz', 2)])
a.join(b).collect()

In [ ]:
r = sc.parallelize([(1, 1), (1, 2), (2, 1)]).groupByKey().collect()
r

In [ ]:
for i, j in r:
    print i, list(j)

## Aggregate

In [ ]:
import numpy as np

In [ ]:
def seqOp((N, partial_sum), value):
    return N + 1, partial_sum + value
def combOp((N1, partial_sum1), (N2, partial_sum2)):
    return N1 + N2, partial_sum1 + partial_sum2

In [ ]:
values = np.random.standard_normal(100)
N, total_sum = sc.parallelize(values).aggregate((0, 0), seqOp, combOp)

In [ ]:
mean = total_sum/N
print mean, values.mean()

# Application: Search using tf-idf

In [ ]:
def tf(term, RDD):
    tlower = term.lower()
    def seqOp((N, term_count), term):
        if term.lower() == tlower:
            return (N+1, term_count+1)
        else:
            return (N+1, term_count)
    def combOp((N1, c1), (N2, c2)):
        return (N1+N2, c1+c2)
    wordsOnly = RDD.filter(lambda x: re.match('\w+', x))
    N, term_count = wordsOnly.aggregate((0, 0), seqOp, combOp)
    return float(term_count) / N

In [ ]:
tf('obama', obamaRDD)

In [ ]:
def makeRDD(url):
    soup = BeautifulSoup(requests.get(url).text, 'lxml')
    for unwanted in soup(['script', 'style']):
        unwanted.decompose()
    text = soup.get_text()
    return sc.parallelize(text.split())

In [ ]:
BushRDD = makeRDD('https://en.wikipedia.org/wiki/George_W._Bush')
TrumpRDD = makeRDD('https://en.wikipedia.org/wiki/Donald_Trump')

In [ ]:
tf('obama', BushRDD)

In [ ]:
tf('obama', TrumpRDD)

In [ ]:
RDDS = [BushRDD, TrumpRDD, obamaRDD]
names = 'Bush Trump Obama'.split()

In [ ]:
for name, RDD in zip(names, RDDS):
    print name, tf('war', RDD)

In [ ]:
for RDD in RDDS:
    print tf('hawaii', RDD)

In [ ]:
for RDD in RDDS:
    print tf('hotels', RDD)

In [ ]:
from math import log

In [ ]:
def idf(term, corpus):
    N = float(len(corpus))
    return log(1 + N / sum(1 for RDD in corpus if tf(term, RDD) > 0))

In [ ]:
def search(term):
    i = idf(term, RDDS)
    for name, RDD in zip(names, RDDS):
        t = tf(term, RDD)
        print name, t*i*100

In [ ]:
search('hotels')

In [ ]:
search('hawaii')

In [ ]:
search('war')

In [ ]:
search('a')

In [ ]:
search('the')

In [ ]:
search('trump')

In [ ]:
search('obama')

## DataFrames

In [ ]:
with open('data/winequality-white.csv') as wine_file:
    header = wine_file.readline()
    wines = sc.parallelize(wine_file.readlines())

In [ ]:
wines.take(5)

In [ ]:
header = [i.strip().replace('"','') for i in header.split(';')]

In [ ]:
header

In [ ]:
from pyspark.sql import Row

In [ ]:
Row(name='Jack Sparrow', rank='Captain')

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("STK-INF4000 White Wine Winners") \
    .getOrCreate()

In [ ]:
(wines
 .map(lambda x: [float(i.strip()) for i in x.split(';')])
 .map(lambda x: Row(**{label: value for label, value in zip(header, x)}))
 .take(2))

In [ ]:
wineDF = spark.createDataFrame(
wines
 .map(lambda x: [float(i.strip()) for i in x.split(';')])
 .map(lambda x: Row(**{label: value for label, value in zip(header, x)}))
)

In [ ]:
wineDF['alcohol', 'quality', 'residual sugar'].show(5)

In [ ]:
wineDF.select(wineDF['alcohol'] + 1).show(5)

In [ ]:
wineDF.filter(wineDF['alcohol'] > 11).select('quality').show(5)

In [ ]:
from pyspark.sql import functions as func

In [ ]:
wineDF.groupBy(wineDF['quality']).agg(func.mean('alcohol'), func.count('quality')).orderBy('quality').show()